In [1]:
!pip3 install tune-sklearn ray[tune]

zsh:1: no matches found: ray[tune]


In [2]:
import sys
import os
import math
sys.path.append('/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction')

from utils.get_data_loader import SustainBenchTextDataset
from sklearn.linear_model import Ridge, LogisticRegression
import numpy as np
# from tune_sklearn import TuneGridSearchCV
import time # Just to compare fit times
from sklearn.metrics import r2_score, classification_report

In [3]:
classification_cutoff_dict = {'asset_index': 0, 'sanitation_index': 3, 'water_index': 3, 'women_edu': 5}
TARGETS = ['asset_index', 'sanitation_index', 'water_index', 'women_edu']

In [4]:
model_type = 'regression'
for target in TARGETS:
    for feature_type in ['target_sentence', 'all_sentence', 'document', 'target_all_sentence', 'target_sentence_document', 'all_sentence_document']:
        ds = SustainBenchTextDataset(
            data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
            feature_type=feature_type, 
            target=target,
            model_type=model_type,
            classification_cutoff=classification_cutoff_dict[target]
        )

        train_embeddings, train_labels = ds.get_data('train')
        test_embeddings, test_labels = ds.get_data('test')
        print(train_embeddings.shape, test_embeddings.shape)

        clf = Ridge()
        parameters = {
           'alpha': np.logspace(0,2,30)
        }
        start = time.time()
        clf.fit(train_embeddings, train_labels)
        end = time.time()
        print("Baseline Fit Time:", end - start)
        
        test_pred = clf.predict(test_embeddings)
        
        print(f'target: {target}, model_type: {model_type}, feature_type: {feature_type}, score: {r2_score(test_labels, test_pred)}')
        print()

(1944, 384) (390, 384)
Baseline Fit Time: 0.014061212539672852
target: asset_index, model_type: regression, feature_type: target_sentence, score: 0.14536840266418138

(5088, 384) (903, 384)
Baseline Fit Time: 0.030302047729492188
target: asset_index, model_type: regression, feature_type: all_sentence, score: 0.13897502204693135

(25200, 300) (7821, 300)
Baseline Fit Time: 0.060365915298461914
target: asset_index, model_type: regression, feature_type: document, score: -0.09272678030906745

(1944, 768) (390, 768)
Baseline Fit Time: 0.040525197982788086
target: asset_index, model_type: regression, feature_type: target_all_sentence, score: 0.21712441737056742

(1944, 684) (390, 684)
Baseline Fit Time: 0.030026912689208984
target: asset_index, model_type: regression, feature_type: target_sentence_document, score: 0.16005229929211484

(5088, 684) (903, 684)
Baseline Fit Time: 0.07065081596374512
target: asset_index, model_type: regression, feature_type: all_sentence_document, score: 0.157580

In [5]:
model_type = 'classification'
for target in TARGETS:
    for feature_type in ['target_sentence', 'all_sentence', 'document', 'target_all_sentence', 'target_sentence_document', 'all_sentence_document']:
        ds = SustainBenchTextDataset(
            data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
            feature_type=feature_type, 
            target=target,
            model_type=model_type,
            classification_cutoff=classification_cutoff_dict[target]
        )

        train_embeddings, train_labels = ds.get_data('train')
        test_embeddings, test_labels = ds.get_data('test')
        print(train_embeddings.shape, test_embeddings.shape)

        clf = LogisticRegression()
        parameters = {
           'C': np.logspace(0,2,30)
        }
        
        clf.fit(train_embeddings, train_labels)
        test_pred = clf.predict(test_embeddings)

        print(f'target: {target}, model_type: {model_type}, feature_type: {feature_type}')
        print(classification_report(test_labels, test_pred))
        print()

(1944, 384) (390, 384)
target: asset_index, model_type: classification, feature_type: target_sentence
              precision    recall  f1-score   support

           0       0.63      0.94      0.75       235
           1       0.64      0.16      0.26       155

    accuracy                           0.63       390
   macro avg       0.64      0.55      0.51       390
weighted avg       0.63      0.63      0.56       390


(5088, 384) (903, 384)
target: asset_index, model_type: classification, feature_type: all_sentence
              precision    recall  f1-score   support

           0       0.61      0.94      0.74       485
           1       0.82      0.31      0.45       418

    accuracy                           0.65       903
   macro avg       0.72      0.62      0.59       903
weighted avg       0.71      0.65      0.60       903


(25200, 300) (7821, 300)
target: asset_index, model_type: classification, feature_type: document
              precision    recall  f1-score   

/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(5088, 684) (903, 684)


/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


target: asset_index, model_type: classification, feature_type: all_sentence_document
              precision    recall  f1-score   support

           0       0.59      0.83      0.69       485
           1       0.64      0.34      0.44       418

    accuracy                           0.60       903
   macro avg       0.61      0.59      0.57       903
weighted avg       0.61      0.60      0.58       903


(2452, 384) (597, 384)
target: sanitation_index, model_type: classification, feature_type: target_sentence
              precision    recall  f1-score   support

           0       0.46      0.91      0.61       275
           1       0.54      0.09      0.16       322

    accuracy                           0.47       597
   macro avg       0.50      0.50      0.38       597
weighted avg       0.50      0.47      0.37       597


(5089, 384) (1002, 384)
target: sanitation_index, model_type: classification, feature_type: all_sentence
              precision    recall  f1-score   s

...

/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ca

target: water_index, model_type: classification, feature_type: all_sentence_document
              precision    recall  f1-score   support

           0       0.79      0.95      0.86       696
           1       0.43      0.14      0.21       207

    accuracy                           0.76       903
   macro avg       0.61      0.54      0.54       903
weighted avg       0.71      0.76      0.71       903


(4136, 384) (1096, 384)
target: women_edu, model_type: classification, feature_type: target_sentence
              precision    recall  f1-score   support

           0       0.46      1.00      0.63       506
           1       0.00      0.00      0.00       590

    accuracy                           0.46      1096
   macro avg       0.23      0.50      0.32      1096
weighted avg       0.21      0.46      0.29      1096


(6282, 384) (1641, 384)
target: women_edu, model_type: classification, feature_type: all_sentence
              precision    recall  f1-score   support

     

/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


target: women_edu, model_type: classification, feature_type: target_sentence_document
              precision    recall  f1-score   support

           0       0.47      0.93      0.63       506
           1       0.64      0.11      0.19       590

    accuracy                           0.49      1096
   macro avg       0.56      0.52      0.41      1096
weighted avg       0.56      0.49      0.39      1096


(6282, 684) (1641, 684)
target: women_edu, model_type: classification, feature_type: all_sentence_document
              precision    recall  f1-score   support

           0       0.42      0.93      0.58       678
           1       0.65      0.09      0.16       963

    accuracy                           0.44      1641
   macro avg       0.54      0.51      0.37      1641
weighted avg       0.56      0.44      0.33      1641




/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/venvs/sustainbench/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [6]:
# model_type = 'regression'
# for target in ['asset_index', 'sanitation_index', 'women_edu']:
#     for feature_type in ['target_sentence', 'all_sentence', 'document', 'target_all_sentence', 'target_sentence_document', 'all_sentence_document']:
#         ds = SustainBenchTextDataset(
#             data_dir=f'/Users/caravanuden/git-repos/Multimodal-deep-learning-for-poverty-prediction/data/', 
#             feature_type=feature_type, 
#             target=target,
#             model_type=model_type,
#             classification_cutoff=classification_cutoff_dict[target]
#         )

#         train_embeddings, train_labels = ds.get_data('train')
#         test_embeddings, test_labels = ds.get_data('test')
#         print(train_embeddings.shape, test_embeddings.shape)

#         clf = Ridge()
#         parameters = {
#            'alpha': np.logspace(0,2,30)
#         }
#         start = time.time()
#         clf.fit(train_embeddings, train_labels)
#         end = time.time()
#         print("Baseline Fit Time:", end - start)
        
#         tune_search = TuneGridSearchCV(
#             Ridge(),
#             parameters,
#             early_stopping=False,
#             max_iters=10
#         )
#         start = time.time()
#         tune_search.fit(X_train, y_train)
#         end = time.time()
#         print("Tune Fit Time:", end - start)
#         pred = tune_search.predict(X_test)
        
#         accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
#         print("Tune Accuracy:", accuracy)

In [7]:
# from hypopt import GridSearch
# from sklearn.linear_model import Ridge

# # Grid-search all parameter combinations using a validation set
# gs = GridSearch(model = Ridge(), param_grid=param_grid, parallelize=False)

# # scoring = None will default to optimizing model.score()
# _ = gs.fit(train_embeddings, train_labels, val_embeddings, val_labels)